In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys


sys.path.insert(0, "..")   

from src.ML_benchmark_prediction_analysis import (
    metrics_per_category
)

In [16]:
%reload_ext autoreload
%autoreload 2

In [44]:
path = Path("../output/data/20251031_all_binding_db_genes.parquet")
df = pd.read_parquet(path)
df.head()

,Drug_Name,Drug,Target Name,Target,Target_Class,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,Mutant,True_Affinity,Predicted_Affinity,Model,Dataset,Gene,Role
0,CHEMBL4209679,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,None,None,None,None,Mutant,4.568636,5.089439,ESMv1,random,None,None
1,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Enzyme,Enzyme,Kinase,Protein Kinase,Mutant,5.000000,5.154931,ESMv1,random,None,None
2,"US11248001, Example 467",CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Enzyme,Enzyme,Protease,Serine protease,Mutant,6.698970,6.506505,ESMv1,random,None,None
3,AZD-6244::CHEMBL1614701,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(cc12)C(=O)NOCCO,Myosin-IIIa,MFPLIGKTIIFDNFPDPSDTWEITETIGKGTYGKVFKVLNKKNGQK...,None,None,None,None,No match,5.000000,5.000000,ESMv1,random,None,None
4,PKC-412::cid_24202429,CO[C@H]1[C@H](C[C@@H]2O[C@@]1(C)n1c3ccccc3c3c4...,Macrophage colony-stimulating factor 1 receptor,MGPGVLLLLLVATAWHGQGIPVIEPSVPELVVKPGATVTLRCVGNG...,Enzyme,Enzyme,Kinase,Protein Kinase,Mutant,6.481486,6.542833,ESMv1,random,CSF1R,Oncogene


In [39]:
df.columns

Index(['Drug_Name', 'Drug', 'Target Name', 'Target', 'Target_Class',
       'Target_Class_Level_1', 'Target_Class_Level_2', 'Target_Class_Level_3',
       'Mutant', 'True_Affinity', 'Predicted_Affinity', 'Model', 'Dataset',
       'Gene', 'Role'],
      dtype='object')

In [41]:
df.shape

(9480588, 15)

In [45]:
print(df["Drug"].nunique())
print(df["Target"].nunique())

18311
1874


In [46]:

def fn(g: pd.DataFrame) -> pd.Series:
    """Calculate metrics for one group, handling small sample sizes"""
    #print(g["Drug_Name"])
    # Check if we have enough data points
    return pd.Series(
        {
            "n": len(g),
        }
    )    

In [49]:
cols = ["Drug", "Target"]
#cols = ["Drug", "Target", "Model", "True_Affinity", "Predicted_Affinity"]

r = df.head(10).groupby(cols, dropna=True).apply(lambda g: fn(g), include_groups=False).reset_index()

In [50]:
r

,Drug,Target,n
0,CO[C@H]1[C@H](C[C@@H]2O[C@@]1(C)n1c3ccccc3c3c4...,MGPGVLLLLLVATAWHGQGIPVIEPSVPELVVKPGATVTLRCVGNG...,6
1,CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,1
2,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(cc12)C(=O)NOCCO,MFPLIGKTIIFDNFPDPSDTWEITETIGKGTYGKVFKVLNKKNGQK...,1
3,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,1
4,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,1


In [37]:
import pandas as pd

# Original dataframe
df = pd.DataFrame({
    'Drug': ['DrugA', 'DrugA', 'DrugA', 'DrugB', 'DrugB'],
    'Target': ['Target1', 'Target1', 'Target2', 'Target1', 'Target1'],
    'Model': ['Model1', 'Model2', 'Model3', 'Model4', 'Model5']
})

def fn(g: pd.DataFrame) -> pd.Series:
    return pd.Series({"n": len(g)})

cols = ["Drug", "Target"]
result = df.groupby(cols, dropna=True).apply(fn, include_groups=True)

print("Original df (unchanged):")
print(df)
print("\nResult (new aggregated data):")
print(result)

Original df (unchanged):
    Drug   Target   Model
0  DrugA  Target1  Model1
1  DrugA  Target1  Model2
2  DrugA  Target2  Model3
3  DrugB  Target1  Model4
4  DrugB  Target1  Model5

Result (new aggregated data):
               n
Drug  Target    
DrugA Target1  2
      Target2  1
DrugB Target1  2


/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_54365/3070940911.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(cols, dropna=True).apply(fn, include_groups=True)
